# Preliminary Model

In this notebook we go through the data to understand it, converted it into standart form which easier is to deal with, run basic models.

This notebook is implemented on local machine.
We have completed Infra stepup(docerization and storing data on cloud), and we will be moving our model onto that in next step.

## Data Specification:

### SQuAD (Standford Question Answer Data Set) 1.0
    1)Questions Answer dataset, with question posed by crowdworker on wikipedia article.
    2)536 articles(Slip into Train, Dev, Test)
    3)107,785 question-answer pairs.


In [1]:
# Importing Required Libraries
import warnings
warnings.filterwarnings('ignore')
import pickle
import numpy as np
import pandas as pd
import json
from textblob import TextBlob
import nltk
from scipy import spatial
import torch
import spacy
en_nlp = spacy.load('en_core_web_sm')

In [152]:
import ast 
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn import linear_model
from sklearn import metrics
from sklearn.model_selection import train_test_split

Reading data from Json file from local machine

In [2]:
train = pd.read_json("data/train-v1.1.json")

In [3]:
valid = pd.read_json("data/dev-v1.1.json")

In [4]:
# Detail of data
train.shape, valid.shape

((442, 2), (48, 2))

Data include 422 articles for training and 48 for validiation with multiple question answer for each

In [5]:
train.head(3)

,data,version
0,"{'title': 'University_of_Notre_Dame', 'paragra...",1.1
1,"{'title': 'Beyoncé', 'paragraphs': [{'context'...",1.1
2,"{'title': 'Montana', 'paragraphs': [{'context'...",1.1


In [14]:
valid.head(3)

,data,version
0,"{'title': 'Super_Bowl_50', 'paragraphs': [{'co...",1.1
1,"{'title': 'Warsaw', 'paragraphs': [{'context':...",1.1
2,"{'title': 'Normans', 'paragraphs': [{'context'...",1.1


In [21]:
# Individual Article Look like this
train.iloc[1,0]

{'title': 'Beyoncé',
 'paragraphs': [{'context': 'Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny\'s Child. Managed by her father, Mathew Knowles, the group became one of the world\'s best-selling girl groups of all time. Their hiatus saw the release of Beyoncé\'s debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy".',
   'qas': [{'answers': [{'answer_start': 269, 'text': 'in the late 1990s'}],
     'question': 'When did Beyonce start becoming popular?',
     'id': '56be85543aeaaa14008c9063'},
    {'answers': [{'answer_start': 207, 'text': 'singing and dancing'}],


In [29]:
train.iloc[1,0]['title']

'Beyoncé'

In [26]:
train.iloc[1,0]['paragraphs'][0]

{'context': 'Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny\'s Child. Managed by her father, Mathew Knowles, the group became one of the world\'s best-selling girl groups of all time. Their hiatus saw the release of Beyoncé\'s debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy".',
 'qas': [{'answers': [{'answer_start': 269, 'text': 'in the late 1990s'}],
   'question': 'When did Beyonce start becoming popular?',
   'id': '56be85543aeaaa14008c9063'},
  {'answers': [{'answer_start': 207, 'text': 'singing and dancing'}],
   'question': 'What areas did Beyonce compet

In [27]:
train.iloc[1,0]['paragraphs'][1]

{'context': 'Following the disbandment of Destiny\'s Child in June 2005, she released her second solo album, B\'Day (2006), which contained hits "Déjà Vu", "Irreplaceable", and "Beautiful Liar". Beyoncé also ventured into acting, with a Golden Globe-nominated performance in Dreamgirls (2006), and starring roles in The Pink Panther (2006) and Obsessed (2009). Her marriage to rapper Jay Z and portrayal of Etta James in Cadillac Records (2008) influenced her third album, I Am... Sasha Fierce (2008), which saw the birth of her alter-ego Sasha Fierce and earned a record-setting six Grammy Awards in 2010, including Song of the Year for "Single Ladies (Put a Ring on It)". Beyoncé took a hiatus from music in 2010 and took over management of her career; her fourth album 4 (2011) was subsequently mellower in tone, exploring 1970s funk, 1980s pop, and 1990s soul. Her critically acclaimed fifth studio album, Beyoncé (2013), was distinguished from previous releases by its experimental production an

### Converting Json Data into dataframe

In [6]:
contexts = []
questions = []
answers_text = []
answers_start = []
for i in range(train.shape[0]):
    topic = train.iloc[i,0]['paragraphs']
    for sub_para in topic:
        for q_a in sub_para['qas']:
            questions.append(q_a['question'])
            answers_start.append(q_a['answers'][0]['answer_start'])
            answers_text.append(q_a['answers'][0]['text'])
            contexts.append(sub_para['context'])   
df = pd.DataFrame({"context":contexts, "question": questions, "answer_start": answers_start, "text": answers_text})

In [7]:
df.shape

(87599, 4)

In [32]:
df.head(5)

,context,question,answer_start,text
0,"Architecturally, the school has a Catholic cha...",To whom did the Virgin Mary allegedly appear i...,515,Saint Bernadette Soubirous
1,"Architecturally, the school has a Catholic cha...",What is in front of the Notre Dame Main Building?,188,a copper statue of Christ
2,"Architecturally, the school has a Catholic cha...",The Basilica of the Sacred heart at Notre Dame...,279,the Main Building
3,"Architecturally, the school has a Catholic cha...",What is the Grotto at Notre Dame?,381,a Marian place of prayer and reflection
4,"Architecturally, the school has a Catholic cha...",What sits on top of the Main Building at Notre...,92,a golden statue of the Virgin Mary


In [8]:
df.to_csv("data/train.csv", index = None)

### Model 1 
(Using basic text analytics techniques, Equlidian Distance and Cosine similarity for answer prediction)

In [9]:
paras = list(df["context"].drop_duplicates().reset_index(drop= True))
len(paras)

18891

In [10]:
blob = TextBlob(" ".join(paras))
sentences = [item.raw for item in blob.sentences]
len(sentences)

92659

In [11]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to C:\Users\Sarmad-
[nltk_data]     PC\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

 Using Infersent for sentence embeding ("https://github.com/facebookresearch/InferSent")

In [12]:

from models import InferSent
V = 2
MODEL_PATH = 'InferSent/encoder/infersent%s.pkl' % V
params_model = {'bsize': 64, 'word_emb_dim': 300, 'enc_lstm_dim': 2048,
                'pool_type': 'max', 'dpout_model': 0.0, 'version': V}
infersent = InferSent(params_model)
infersent.load_state_dict(torch.load(MODEL_PATH))

<All keys matched successfully>

In [14]:
W2V_PATH = 'InferSent/dataset/glove.840B.300d.txt'
infersent.set_w2v_path(W2V_PATH)

In [15]:
infersent.build_vocab(sentences, tokenize=True)

Found 89008(/109703) words with w2v vectors
Vocab size : 89008


In [17]:
dict_embeddings = {}
for i in range(len(sentences)):
    #print(i) 
    dict_embeddings[sentences[i]] = infersent.encode([sentences[i]], tokenize=True)
questions = list(df["question"])
len(questions)

87599

In [19]:
for i in range(len(questions)):
    #print(i)
    dict_embeddings[questions[i]] = infersent.encode([questions[i]], tokenize=True)

Using pre trained model of infresent, we embed our sentence, and saving learned pickle

In [23]:
dict_embeddings['Architecturally, the school has a Catholic character. has'][0]

array([ 0.01158243,  0.06023026, -0.05040139, ...,  0.0940277 ,
        0.04296588,  0.09574181], dtype=float32)

In [24]:
with open('data/dict_embeddings.pickle', 'wb') as handle:
    pickle.dump(dict_embeddings, handle)

In [26]:
dict_emb = dict_embeddings

In [57]:
del dict_embeddings

In [58]:
len(dict_emb)

179862

In [30]:
df.head(3)

,context,question,answer_start,text
0,"Architecturally, the school has a Catholic cha...",To whom did the Virgin Mary allegedly appear i...,515,Saint Bernadette Soubirous
1,"Architecturally, the school has a Catholic cha...",What is in front of the Notre Dame Main Building?,188,a copper statue of Christ
2,"Architecturally, the school has a Catholic cha...",The Basilica of the Sacred heart at Notre Dame...,279,the Main Building


In [32]:
df.dropna(inplace=True)
df.shape

(87599, 4)

In [33]:
def get_target(x):
    idx = -1
    for i in range(len(x["sentences"])):
        if x["text"] in x["sentences"][i]: idx = i
    return idx

In [34]:
## Routine for applying embeded  sentences to dataframe 
def process_data(train):
    
    print("step 1")
    train['sentences'] = train['context'].apply(lambda x: [item.raw for item in TextBlob(x).sentences])
    
    print("step 2")
    train["target"] = train.apply(get_target, axis = 1)
    
    print("step 3")
    train['sent_emb'] = train['sentences'].apply(lambda x: [dict_emb[item][0] if item in\
                                                           dict_emb else np.zeros(4096) for item in x])
    print("step 4")
    train['quest_emb'] = train['question'].apply(lambda x: dict_emb[x] if x in dict_emb else np.zeros(4096) )
        
    return train 

In [39]:
df = process_data(df)

step 1
step 2
step 3
step 4


In [40]:
df.head(5)

,context,question,answer_start,text,sentences,target,sent_emb,quest_emb
0,"Architecturally, the school has a Catholic cha...",To whom did the Virgin Mary allegedly appear i...,515,Saint Bernadette Soubirous,"[Architecturally, the school has a Catholic ch...",5,"[[0.011582431, 0.06023026, -0.050401386, -0.01...","[[0.014639893, 0.1659823, 0.11722623, 0.026021..."
1,"Architecturally, the school has a Catholic cha...",What is in front of the Notre Dame Main Building?,188,a copper statue of Christ,"[Architecturally, the school has a Catholic ch...",2,"[[0.011582431, 0.06023026, -0.050401386, -0.01...","[[0.04561126, 0.26295376, 0.087074116, 0.03128..."
2,"Architecturally, the school has a Catholic cha...",The Basilica of the Sacred heart at Notre Dame...,279,the Main Building,"[Architecturally, the school has a Catholic ch...",3,"[[0.011582431, 0.06023026, -0.050401386, -0.01...","[[-0.015232063, 0.25448513, 0.13147594, -0.009..."
3,"Architecturally, the school has a Catholic cha...",What is the Grotto at Notre Dame?,381,a Marian place of prayer and reflection,"[Architecturally, the school has a Catholic ch...",4,"[[0.011582431, 0.06023026, -0.050401386, -0.01...","[[0.04561126, 0.29053542, 0.06631295, 0.031285..."
4,"Architecturally, the school has a Catholic cha...",What sits on top of the Main Building at Notre...,92,a golden statue of the Virgin Mary,"[Architecturally, the school has a Catholic ch...",1,"[[0.011582431, 0.06023026, -0.050401386, -0.01...","[[0.04561126, 0.29014665, 0.13065585, 0.072881..."


In [35]:
# Method for predicting output using cosine similarity

def cosine_sim(x):
    li = []
    for item in x["sent_emb"]:
        li.append(spatial.distance.cosine(item,x["quest_emb"][0]))
    return li  

In [36]:
# Method for predicting output with minimum diastance
def pred_idx(distances):
    return np.argmin(distances)   

In [37]:
# Predicting using euclidean and cosine for whole training data.
def predictions(train):
    
    train["cosine_sim"] = train.apply(cosine_sim, axis = 1)
    train["diff"] = (train["quest_emb"] - train["sent_emb"])**2
    train["euclidean_dis"] = train["diff"].apply(lambda x: list(np.sum(x, axis = 1)))
    del train["diff"]
    
    print("cosine start")
    
    train["pred_idx_cos"] = train["cosine_sim"].apply(lambda x: pred_idx(x))
    train["pred_idx_euc"] = train["euclidean_dis"].apply(lambda x: pred_idx(x))
    
    return train

In [38]:
def accuracy(target, predicted):
    
    acc = (target==predicted).sum()/len(target)
    
    return acc

In [41]:
df_dist = predictions(df)

cosine start


In [42]:
df_dist.head(4)

,context,question,answer_start,text,sentences,target,sent_emb,quest_emb,cosine_sim,euclidean_dis,pred_idx_cos,pred_idx_euc
0,"Architecturally, the school has a Catholic cha...",To whom did the Virgin Mary allegedly appear i...,515,Saint Bernadette Soubirous,"[Architecturally, the school has a Catholic ch...",5,"[[0.011582431, 0.06023026, -0.050401386, -0.01...","[[0.014639893, 0.1659823, 0.11722623, 0.026021...","[0.22806447744369507, 0.18822699785232544, 0.1...","[10.346078, 12.060574, 15.814236, 9.22311, 8.2...",5,5
1,"Architecturally, the school has a Catholic cha...",What is in front of the Notre Dame Main Building?,188,a copper statue of Christ,"[Architecturally, the school has a Catholic ch...",2,"[[0.011582431, 0.06023026, -0.050401386, -0.01...","[[0.04561126, 0.26295376, 0.087074116, 0.03128...","[0.21697574853897095, 0.1409621238708496, 0.16...","[10.452, 9.261847, 13.540291, 6.7479477, 8.372...",3,3
2,"Architecturally, the school has a Catholic cha...",The Basilica of the Sacred heart at Notre Dame...,279,the Main Building,"[Architecturally, the school has a Catholic ch...",3,"[[0.011582431, 0.06023026, -0.050401386, -0.01...","[[-0.015232063, 0.25448513, 0.13147594, -0.009...","[0.222112774848938, 0.1619478464126587, 0.1889...","[10.053694, 10.526111, 15.297926, 6.1321497, 5...",4,4
3,"Architecturally, the school has a Catholic cha...",What is the Grotto at Notre Dame?,381,a Marian place of prayer and reflection,"[Architecturally, the school has a Catholic ch...",4,"[[0.011582431, 0.06023026, -0.050401386, -0.01...","[[0.04561126, 0.29053542, 0.06631295, 0.031285...","[0.23786461353302002, 0.23242729902267456, 0.2...","[9.982977, 14.521368, 19.81585, 9.85925, 7.429...",4,4


In [48]:
def accuracy(target, predicted):
    
    acc = (target==predicted).sum()/len(target)
    
    return acc

In [50]:
print("Accuracy of Prediction Using Eucledian Distance",accuracy(df_dist["target"], df_dist["pred_idx_euc"]))

Accuracy of Prediction Using Eucledian Distance 0.4004383611685065


In [51]:
print("Accuracy of Prediction Using Cosine Similarity",accuracy(df_dist["target"], df_dist["pred_idx_cos"]))

Accuracy of Prediction Using Cosine Similarity 0.4905877920980833


In [53]:
df_dist.to_csv("data/trained.csv", index=None)

In [54]:
df_dist.iloc[323,:]

context          Beyoncé Giselle Knowles was born in Houston, T...
question             What is the name of Beyoncé's younger sister?
answer_start                                                   255
text                                                       Solange
sentences        [Beyoncé Giselle Knowles was born in Houston, ...
target                                                           2
sent_emb         [[-0.018363085, 0.1571888, 0.11054845, 0.00331...
quest_emb        [[0.04561126, 0.09627831, 0.09477972, 0.042733...
cosine_sim       [0.21232259273529053, 0.09829854965209961, 0.1...
euclidean_dis    [12.700025, 4.3109384, 5.2169576, 13.662064, 8...
pred_idx_cos                                                     1
pred_idx_euc                                                     1
Name: 323, dtype: object

In [56]:
df_dist.iloc[12313,:]

context          Inns are buildings where travellers can seek l...
question                   What road is an inn often located near?
answer_start                                                   136
text                                                       highway
sentences        [Inns are buildings where travellers can seek ...
target                                                           1
sent_emb         [[0.00369396, 0.14177673, 0.020265581, 0.05489...
quest_emb        [[0.04561126, 0.20803837, 0.023483012, 0.03128...
cosine_sim       [0.16052162647247314, 0.1060645580291748, 0.19...
euclidean_dis    [8.409194, 4.6776714, 9.447313, 9.124254, 10.4...
pred_idx_cos                                                     1
pred_idx_euc                                                     1
Name: 12313, dtype: object

In [71]:
del df_dist

## Model 2 Applying Regression

In [72]:
data = pd.read_csv("data/trained.csv").reset_index(drop=True)

In [73]:
ast.literal_eval(data["sentences"][0])

['Architecturally, the school has a Catholic character.',
 "Atop the Main Building's gold dome is a golden statue of the Virgin Mary.",
 'Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes".',
 'Next to the Main Building is the Basilica of the Sacred Heart.',
 'Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection.',
 'It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858.',
 'At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.']

In [74]:
data = data[data["sentences"].apply(lambda x: len(ast.literal_eval(x)))<11].reset_index(drop=True)

In [75]:
# Making Euclidian and Cosine as feature Vector into a training dataframe
def create_features(data):
    train = pd.DataFrame()
     
    for k in range(len(data["euclidean_dis"])):
        dis = ast.literal_eval(data["euclidean_dis"][k])
        for i in range(len(dis)):
            train.loc[k, "column_euc_"+"%s"%i] = dis[i]
    
    for k in range(len(data["cosine_sim"])):
        dis = ast.literal_eval(data["cosine_sim"][k].replace("nan","1"))
        for i in range(len(dis)):
            train.loc[k, "column_cos_"+"%s"%i] = dis[i]
            
    train["target"] = data["target"]
    print("Finished")
    return train

In [76]:
train = create_features(data)

Finished


In [87]:
train.head(4)

,column_euc_0,column_euc_1,column_euc_2,column_euc_3,column_euc_4,column_euc_5,column_euc_6,column_euc_7,column_euc_8,column_euc_9,...,column_cos_1,column_cos_2,column_cos_3,column_cos_4,column_cos_5,column_cos_6,column_cos_7,column_cos_8,column_cos_9,target
0,10.346078,12.060574,15.814236,9.223110,8.238199,7.088388,8.668980,NaN,NaN,NaN,...,0.188227,0.196962,0.184189,0.173519,0.121387,0.166792,NaN,NaN,NaN,5
1,10.452000,9.261847,13.540291,6.747948,8.372871,11.238192,8.238443,NaN,NaN,NaN,...,0.140962,0.165420,0.129633,0.167571,0.193154,0.153606,NaN,NaN,NaN,2
2,10.053694,10.526111,15.297926,6.132150,5.801221,9.142113,6.573022,NaN,NaN,NaN,...,0.161948,0.188913,0.122453,0.122407,0.158934,0.126132,NaN,NaN,NaN,3
3,9.982977,14.521368,19.815850,9.859250,7.429172,11.572577,10.106126,NaN,NaN,NaN,...,0.232427,0.256629,0.206010,0.166145,0.207057,0.201885,NaN,NaN,NaN,4


In [140]:
# Dealing with missing values
subset1 = train.iloc[:,:10]
subset2 = train.iloc[:,10:20]
subset2.head(3)

,column_cos_0,column_cos_1,column_cos_2,column_cos_3,column_cos_4,column_cos_5,column_cos_6,column_cos_7,column_cos_8,column_cos_9
0,0.228064,0.188227,0.196962,0.184189,0.173519,0.121387,0.166792,NaN,NaN,NaN
1,0.216976,0.140962,0.165420,0.129633,0.167571,0.193154,0.153606,NaN,NaN,NaN
2,0.222113,0.161948,0.188913,0.122453,0.122407,0.158934,0.126132,NaN,NaN,NaN


In [141]:
subset1  = subset1.apply(lambda row: row.fillna(row.mean()), axis=1)
subset2  = subset2.apply(lambda row: row.fillna(row.mean()), axis=1)
subset2.head(3)

,column_cos_0,column_cos_1,column_cos_2,column_cos_3,column_cos_4,column_cos_5,column_cos_6,column_cos_7,column_cos_8,column_cos_9
0,0.228064,0.188227,0.196962,0.184189,0.173519,0.121387,0.166792,0.179877,0.179877,0.179877
1,0.216976,0.140962,0.165420,0.129633,0.167571,0.193154,0.153606,0.166760,0.166760,0.166760
2,0.222113,0.161948,0.188913,0.122453,0.122407,0.158934,0.126132,0.157557,0.157557,0.157557


In [145]:
train2 = pd.concat([subset1, subset2, train.iloc[:,20]],axis=1)
train2.head(4)

,column_euc_0,column_euc_1,column_euc_2,column_euc_3,column_euc_4,column_euc_5,column_euc_6,column_euc_7,column_euc_8,column_euc_9,...,column_cos_1,column_cos_2,column_cos_3,column_cos_4,column_cos_5,column_cos_6,column_cos_7,column_cos_8,column_cos_9,target
0,10.346078,12.060574,15.814236,9.223110,8.238199,7.088388,8.668980,10.205652,10.205652,10.205652,...,0.188227,0.196962,0.184189,0.173519,0.121387,0.166792,0.179877,0.179877,0.179877,5
1,10.452000,9.261847,13.540291,6.747948,8.372871,11.238192,8.238443,9.693085,9.693085,9.693085,...,0.140962,0.165420,0.129633,0.167571,0.193154,0.153606,0.166760,0.166760,0.166760,2
2,10.053694,10.526111,15.297926,6.132150,5.801221,9.142113,6.573022,9.075177,9.075177,9.075177,...,0.161948,0.188913,0.122453,0.122407,0.158934,0.126132,0.157557,0.157557,0.157557,3
3,9.982977,14.521368,19.815850,9.859250,7.429172,11.572577,10.106126,11.898188,11.898188,11.898188,...,0.232427,0.256629,0.206010,0.166145,0.207057,0.201885,0.215431,0.215431,0.215431,4


In [146]:
train2.apply(max, axis = 0)

column_euc_0    1217.942100
column_euc_1    1217.942100
column_euc_2    1217.942100
column_euc_3    1217.942100
column_euc_4    1217.942100
column_euc_5    1217.942100
column_euc_6    1217.942100
column_euc_7    1217.942100
column_euc_8    1217.942100
column_euc_9    1217.942100
column_cos_0       1.000000
column_cos_1       1.070920
column_cos_2       1.000000
column_cos_3       1.036805
column_cos_4       1.000000
column_cos_5       1.000000
column_cos_6       1.083527
column_cos_7       1.014868
column_cos_8       1.014868
column_cos_9       1.014868
target             9.000000
dtype: float64

In [153]:
scaler = MinMaxScaler()
X = scaler.fit_transform(train2.iloc[:,:-1])
train_x, test_x, train_y, test_y = train_test_split(X,
train.iloc[:,-1], train_size=0.8, random_state = 5)

In [154]:
mul_lr = linear_model.LogisticRegression(multi_class='multinomial', solver='newton-cg')
mul_lr.fit(train_x, train_y)

print("Multinomial Logistic regression Train Accuracy : ", metrics.accuracy_score(train_y, mul_lr.predict(train_x)))
print("Multinomial Logistic regression Test Accuracy : ", metrics.accuracy_score(test_y, mul_lr.predict(test_x)))


Multinomial Logistic regression Train Accuracy :  0.4276022086466165
Multinomial Logistic regression Test Accuracy :  0.4317434210526316


In [161]:
rf = RandomForestClassifier(min_samples_leaf=8, n_estimators=60)
rf.fit(train_x, train_y)

print("Random Forest Train Accuracy : ", metrics.accuracy_score(train_y, rf.predict(train_x)))
print("Random Forest  Test Accuracy : ", metrics.accuracy_score(test_y, rf.predict(test_x)))

Random Forest Train Accuracy :  0.7173695958646616
Random Forest  Test Accuracy :  0.5330122180451128


In [158]:
model = xgb.XGBClassifier()
param_dist = {"max_depth": [5],
              "min_child_weight" : [1],
              "learning_rate": [0.1],
               }

# run randomized search
grid_search = GridSearchCV(model, param_grid=param_dist, cv = 3, 
                                   verbose=5, n_jobs=-1)
grid_search.fit(train_x, train_y)

Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\externals\joblib\numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:  3.6min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:  3.6min finished


[17:14:09] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


GridSearchCV(cv=3, error_score='raise-deprecating',
       estimator=XGBClassifier(base_score=None, booster=None, colsample_bylevel=None,
       colsample_bynode=None, colsample_bytree=None, gamma=None,
       gpu_id=None, importance_type='gain', interaction_constraints=None,
       learning_rate=None, max_delta_step=None, max_depth=None,
       min_child_w...e,
       tree_method=None, use_label_encoder=True, validate_parameters=None,
       verbosity=None),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'max_depth': [5], 'min_child_weight': [1], 'learning_rate': [0.1]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=5)

In [159]:
grid_search.best_estimator_

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
       importance_type='gain', interaction_constraints='',
       learning_rate=0.1, max_delta_step=0, max_depth=5,
       min_child_weight=1, missing=nan, monotone_constraints='()',
       n_estimators=100, n_jobs=8, num_parallel_tree=1,
       objective='multi:softprob', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=None, subsample=1,
       tree_method='exact', use_label_encoder=True, validate_parameters=1,
       verbosity=None)

In [162]:
xg = xgb.XGBClassifier(max_depth=5)
xg.fit(train_x, train_y)

print("XGBoost Train Accuracy : ", metrics.accuracy_score(train_y, xg.predict(train_x)))
print("XGBoost Test Accuracy : ", metrics.accuracy_score(test_y, xg.predict(test_x)))

[17:35:25] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
XGBoost Train Accuracy :  0.6573073308270677
XGBoost Test Accuracy :  0.5615601503759399


## Summary

Accuracy of Prediction Using Eucledian Distance 0.4004383611685065<br>
Accuracy of Prediction Using Cosine Similarity 0.4905877920980833<br>
<br><br>
Multinomial Logistic regression Train Accuracy :  0.4276022086466165<br>
Multinomial Logistic regression Test Accuracy :  0.4317434210526316<br>
<br><br>
Random Forest Train Accuracy :  0.7173695958646616<br>
Random Forest  Test Accuracy :  0.5330122180451128<br>
<br><br>
XGBoost Train Accuracy :  0.6573073308270677<br>
XGBoost Test Accuracy :  0.5615601503759399

## Way Forward

We will be using BERT (Bidirectional Encoder Representations from Transformers) pre trained model, fine tune it  as per our data and add custom model to it to increase its benchmark qccuracy on our dataset.

The accuracy we achived in our model above is low as many advance techniques are already applied with greater accuracy, but using these models we have learnt and explore our dataset at lower level which will help in fine tuning and enchanging accuracy in later steps